In [1]:
print("OK")

OK


In [2]:
%pwd

'c:\\Users\\Asus user\\OneDrive\\Documents\\VS CODE\\Chatbot\\Medical-Chatbot-Gen-AI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Asus user\\OneDrive\\Documents\\VS CODE\\Chatbot\\Medical-Chatbot-Gen-AI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file('Data/')
#extracted_data

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
len(text_chunks)

7023

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

In [14]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

C:\Users\Asus user\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus user\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [49]:
from dotenv import load_dotenv
load_dotenv()

True

In [50]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [24]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [51]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [29]:

# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [31]:
# Load Existing index 

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [32]:
docsearch

In [33]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [34]:
retrieved_docs = retriever.invoke("What is Diabetes?")

In [35]:
retrieved_docs

[Document(metadata={'page': 434.0, 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 2. 2nd ed.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 1047\nDiabetes mellitus'),
 Document(metadata={'page': 435.0, 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 2. 2nd ed.pdf'}, page_content='glucose in the blood cannot be absorbed into the cells of\nthe body. Symptoms include frequent urination, lethargy,\nexcessive thirst, and hunger. The treatment includes\nchanges in diet, oral medications, and in some cases,\ndaily injections of insulin.\nDescription\nDiabetes mellitus is a chronic disease that causes\nserious health complications including renal (kidney)\nfailure, heart disease,stroke, and blindness. Approxi-\nmately 14 million Americans (about 5% of the popula-'),
 Document(metadata={'page': 150.0, 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 2. 2nd ed.pdf'}, page_content='that assess psychological functioning, intelligence,\nand/or personality traits.\nType 1 diabete

In [52]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [53]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [54]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [55]:
response = rag_chain.invoke({"input": "what is Diabetes Retinopathy?"})
print(response["answer"])

Diabetic retinopathy is damage to the tiny blood vessels in the retina, the light-sensing tissue at the back of the eye. This damage can cause blurred vision, sudden blindness, or black spots, lines, or flashing lights in the field of vision.  It is a condition caused by diabetes mellitus.

